<a href="https://colab.research.google.com/github/mrdbourke/tensorflow-deep-learning/blob/main/10_time_series_forecasting_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [30]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import layers

from datetime import datetime
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### Import data

In [31]:
crypto_df = pd.read_csv("../input/g-research-crypto-forecasting/train.csv") 

In [32]:
crypto_df.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [33]:
asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')

In [34]:
asset_details

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [35]:
# Select Asset_ID = 6 for Ethereum
crypto_df = crypto_df[crypto_df["Asset_ID"]==6] 
crypto_df.info(show_counts =True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956200 entries, 5 to 24236799
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   timestamp  1956200 non-null  int64  
 1   Asset_ID   1956200 non-null  int64  
 2   Count      1956200 non-null  float64
 3   Open       1956200 non-null  float64
 4   High       1956200 non-null  float64
 5   Low        1956200 non-null  float64
 6   Close      1956200 non-null  float64
 7   Volume     1956200 non-null  float64
 8   VWAP       1956200 non-null  float64
 9   Target     1955860 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 164.2 MB


In [36]:
crypto_df.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
5,1514764860,6,173.0,738.3025,746.00,732.51,738.5075,335.987856,738.839291,-0.004809
13,1514764920,6,192.0,738.5075,745.14,732.49,738.2600,232.793141,738.268967,-0.004441
21,1514764980,6,120.0,738.3325,745.12,730.00,737.5025,174.138031,737.994457,-0.004206
29,1514765040,6,156.0,737.2225,744.69,728.93,737.1025,165.383926,737.303631,-0.002205
37,1514765100,6,118.0,736.5300,743.80,727.11,735.7050,193.078039,736.163026,-0.001744


### Preprocess data

In [37]:
df = crypto_df.copy()

In [38]:
# fill missing values 
df = df.reindex(range(df.index[0],df.index[-1]+60,60),method='pad')
df = df.fillna(0)

In [39]:
# rename column timestamp to Date 
df.rename({'timestamp': 'Date'}, axis=1, inplace=True)

# rename Close to Price
df.rename(columns={'Close': 'Price'}, inplace=True)

In [40]:
# timestamp conversion
df.Date = df.Date.apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

In [41]:
# set index
df.set_index('Date', inplace=True)

In [42]:
df.head()

,Asset_ID,Count,Open,High,Low,Price,Volume,VWAP,Target
Date,,,,,,,,,
2018-01-01,6,173.0,738.3025,746.00,732.51,738.5075,335.987856,738.839291,-0.004809
2018-01-01,6,111.0,734.2300,744.03,724.43,735.0900,125.164340,734.399631,-0.004800
2018-01-01,6,97.0,735.1225,744.30,725.32,734.8025,104.518346,735.085802,-0.001759
2018-01-01,6,247.0,732.6000,741.48,722.91,731.8200,465.550694,731.937561,0.000960
2018-01-01,6,92.0,732.7950,739.47,724.44,732.9325,43.809790,732.882372,-0.002175


In [43]:
# Convert to date array
timesteps = df.index.to_numpy()
prices = df['Price'].to_numpy()

timesteps[:10], prices[:10]

(array(['2018-01-01', '2018-01-01', '2018-01-01', '2018-01-01',
        '2018-01-01', '2018-01-01', '2018-01-01', '2018-01-01',
        '2018-01-01', '2018-01-01'], dtype=object),
 array([738.5075, 735.09  , 734.8025, 731.82  , 732.9325, 732.3425,
        731.8225, 732.1325, 732.605 , 729.415 ]))

### Modeling: Recurrent Neural Network LSTM

In [44]:
HORIZON = 1 
WINDOW_SIZE = 7

In [45]:
# Function to create labelled window data
def get_labelled_windows(x, horizon=1):
  """
  Creates labels for windowed dataset.
  E.g. if horizon=1 (default)
  Input: [1, 2, 3, 4, 5, 6] -> Output: ([1, 2, 3, 4, 5], [6])
  """
  return x[:, :-horizon], x[:, -horizon:]

In [46]:
# Test the window labelling function
test_window, test_label = get_labelled_windows(tf.expand_dims(tf.range(8)+1, axis=0), horizon=HORIZON)
print(f"Window: {tf.squeeze(test_window).numpy()} -> Label: {tf.squeeze(test_label).numpy()}")

Window: [1 2 3 4 5 6 7] -> Label: 8


In [47]:
# Function to view NumPy arrays as windows 
def make_windows(x, window_size=7, horizon=1):
  """
  Turns a 1D array into a 2D array of sequential windows of window_size.
  """
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T 
  windowed_array = x[window_indexes]
  windows, labels = get_labelled_windows(windowed_array, horizon=horizon)
  return windows, labels

In [48]:
full_windows, full_labels = make_windows(prices, window_size=WINDOW_SIZE, horizon=HORIZON)
len(full_windows), len(full_labels)

(403941, 403941)

In [49]:
# View the first 3 windows/labels
for i in range(3):
  print(f"Window: {full_windows[i]} -> Label: {full_labels[i]}")

Window: [738.5075 735.09   734.8025 731.82   732.9325 732.3425 731.8225] -> Label: [732.1325]
Window: [735.09   734.8025 731.82   732.9325 732.3425 731.8225 732.1325] -> Label: [732.605]
Window: [734.8025 731.82   732.9325 732.3425 731.8225 732.1325 732.605 ] -> Label: [729.415]


In [50]:
# View the last 3 windows/labels
for i in range(3):
  print(f"Window: {full_windows[i-3]} -> Label: {full_labels[i-3]}")

Window: [2954.52428571 2964.53460045 2950.26285714 2969.77714286 2959.72714286
 2955.75428571 2971.90571429] -> Label: [2977.38714286]
Window: [2964.53460045 2950.26285714 2969.77714286 2959.72714286 2955.75428571
 2971.90571429 2977.38714286] -> Label: [2960.84571429]
Window: [2950.26285714 2969.77714286 2959.72714286 2955.75428571 2971.90571429
 2977.38714286 2960.84571429] -> Label: [2972.60333333]


In [51]:
# Function to create train-test-splits
def make_train_test_splits(windows, labels, test_split=0.2):
  """
  Splits matching pairs of windows and labels into train and test splits.
  """
  split_size = int(len(windows) * (1-test_split))
  train_windows = windows[:split_size]
  train_labels = labels[:split_size]
  test_windows = windows[split_size:]
  test_labels = labels[split_size:]
  return train_windows, test_windows, train_labels, test_labels

In [52]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(full_windows, full_labels)
len(train_windows), len(test_windows), len(train_labels), len(test_labels)

(323152, 80789, 323152, 80789)

In [53]:
train_windows[:5], train_labels[:5]

(array([[738.5075, 735.09  , 734.8025, 731.82  , 732.9325, 732.3425,
         731.8225],
        [735.09  , 734.8025, 731.82  , 732.9325, 732.3425, 731.8225,
         732.1325],
        [734.8025, 731.82  , 732.9325, 732.3425, 731.8225, 732.1325,
         732.605 ],
        [731.82  , 732.9325, 732.3425, 731.8225, 732.1325, 732.605 ,
         729.415 ],
        [732.9325, 732.3425, 731.8225, 732.1325, 732.605 , 729.415 ,
         731.32  ]]),
 array([[732.1325],
        [732.605 ],
        [729.415 ],
        [731.32  ],
        [733.5625]]))

In [54]:
import os

# Function to implement a ModelCheckpoint callback with a specific filename 
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), 
                                            verbose=0, 
                                            save_best_only=True)

In [55]:
tf.random.set_seed(42)

# LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) 
x = layers.LSTM(128, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
lstm_model = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

# Compile model
lstm_model.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam())

# Fit the model
lstm_model.fit(train_windows,
            train_labels,
            epochs=100,
            verbose=0,
            batch_size=128,
            validation_data=(test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name=lstm_model.name)])

In [57]:
# Load in best version of the LSTM model
lstm_model = tf.keras.models.load_model("model_experiments/model_5_lstm/")
lstm_model.evaluate(test_windows, test_labels)

2525/2525 [==============================] - 4s 2ms/step - loss: 5.3807


5.38067626953125

In [58]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast)

In [59]:
# Make predictions with our LSTM model
model_lstm_preds = make_preds(lstm_model, test_windows)
model_lstm_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1225.5267, 1231.2714, 1229.4937, 1228.6772, 1227.6976, 1228.045 ,
       1228.3718, 1221.807 , 1225.8633, 1227.8173], dtype=float32)>

### Model Evaluation

In [62]:
def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) 
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  
  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy()}

In [63]:
# Evaluate LSTM model
model_lstm_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                 y_pred=model_lstm_preds)
model_lstm_results

{'mae': 5.3806744, 'mse': 79.15038, 'rmse': 8.89665, 'mape': 0.2246136}